# Figure 2: MS2/Identification visualization

## Library imports

In [1]:
import pandas as pd
import re
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

import utils

## Fig 2 A: MS2 DDA spectra 

For this figure we used the [PXD012867 PRIDE project](https://www.ebi.ac.uk/pride/archive/projects/PXD012867) as example data.

In [2]:
sample_path = '.../Data/PXD012867_yeast_project/20190124_QX3_JuSc_SA_JS7_1_wt_4h_1.raw'

In [3]:
data = utils.load_thermo_raw(sample_path)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 139295/139295 [02:11<00:00, 1061.49it/s]


In [4]:
# this plotting function is taken from the AlphaViz package (https://github.com/MannLabs/alphaviz) and modified
def plot_mass_spectra(
    mz_values: np.ndarray,
    intensity_values: np.ndarray,
    title: str,
    spectrum_color: str = 'grey',
    template: str = "plotly_white",
    spectrum_line_width: float = 1.5,
    height: int = 520
):
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=mz_values,
            y=intensity_values,
            mode='markers',
            marker=dict(color=spectrum_color, size=1),
            hovertemplate='<b>m/z:</b> %{x};<br><b>Intensity:</b> %{y}.',
            name='',
            showlegend=False
        )
    )

    # Use the 'shapes' attribute from the layout to draw the vertical lines
    fig.update_layout(
        template=template,
        shapes=[
            dict(
                type='line',
                xref='x',
                yref='y',
                x0=mz_values[i],
                y0=0,
                x1=mz_values[i],
                y1=intensity_values[i],
                line=dict(
                    color = spectrum_color,
                    width=spectrum_line_width
                )
            ) for i, val in enumerate(zip(mz_values, intensity_values))
        ],
        xaxis=dict(
            title='m/z, Th',
        ),
        yaxis=dict(
            title='Intensity',
        ),
        legend=dict(
            orientation="h",
            x=1,
            xanchor="right",
            yanchor="bottom",
            y=1.01
        ),
        hovermode="closest",
        height=height,
        title=dict(
            text=title,
            font=dict(
                size=16,
            ),
            x=0.5,
            xanchor='center',
            yanchor='top'
        ),
    )

    return fig

In [7]:
df_ms2 = pd.DataFrame({'scan': data['scan_list_ms2'], 
                    'RT': data['rt_list_ms2'],
                    'prec_mono_mz': data['mono_mzs2'],
                    'prec_charge': data['charge_ms2'],
                    'mz_values': data['mass_list_ms2'],
                   'intensity_values': data['int_list_ms2'],
                    'order': 'ms2'})
df_ms2[df_ms2.RT > 50].head()

,scan,RT,prec_mono_mz,prec_charge,mz_values,intensity_values,order
57334,61885,50.001900,590.2973,2,"[100.41511535644531, 101.07136535644531, 101.1...","[7199, 12615, 6445, 21390, 50917, 6285, 11642,...",ms2
57335,61886,50.002591,479.4426,2,"[101.07112121582031, 102.05511474609375, 103.0...","[24542, 28179, 15409, 96100, 8020, 13094, 2720...",ms2
57336,61887,50.003293,732.3805,2,"[101.0713119506836, 102.05506896972656, 109.51...","[14165, 12270, 6441, 34719, 8217, 15476, 7853,...",ms2
57337,61888,50.004004,625.6472,3,"[102.05464172363281, 110.07127380371094, 120.0...","[12502, 37669, 29084, 8254, 8492, 53982, 32433...",ms2
57338,61889,50.004702,517.2747,2,"[101.07125854492188, 102.05510711669922, 110.0...","[17768, 35090, 38741, 5938, 42354, 6035, 23082...",ms2


In [13]:
# let's plot the MS2 spectra acquired at 50.0019 min for the precursor m/z = 590.2973
plot_mass_spectra(
    mz_values=df_ms2[df_ms2.scan == 61885].mz_values.values[0], 
    intensity_values=df_ms2[df_ms2.scan == 61885].intensity_values.values[0],
    title=f"MS2 spectrum for precursor m/z={df_ms2[df_ms2.scan == 61885].prec_mono_mz.values[0]} and charge={df_ms2[df_ms2.scan == 61885].prec_charge.values[0]}."
).show(config=utils.config)